In [3]:
from IPython.display import display,HTML
display(HTML("<style>.container {width:95% !important ; }</style>"))

### 하이퍼 파라미터 튜닝 수행 방법
- Grid Search
- Random Search
- Bayesian Optimization
- 수동 튜닝

### 하이퍼 파라미터 튜닝의 주요 이슈
- Gradient Boosting 기반 알고리즘은 튜닝해야 할 하이퍼 파라미터 개수가 많고 범위가 넓어서 가능한 개별 경우의 수가 너무 많음
- 이러한 경우의 수가 많을 경우 데이터가 크면 하이퍼 파라미터 튜닝에 굉장히 오랜 시간이 투입되어야 함.

### Grid Search와 Random Search의 주요 이슈
- **GridSearchCV**는 수행 시간이 너무 오래 걸림. 개별 하이퍼 파리머티들을 Grid 형태로 지정하는 것은 한계가 존재(데이터 세트가 작을 때 유리)
- **RandomizedSearch**는 수행 시간은 줄여 주지만, Random한 선택으로 최적 하이퍼 파라미터 검출에 태생적 제약(데이터 세트가 클 때 유리)
- 두 가지 방법 모두 iteration 중에 어느정도 최적화된 하이퍼 파라미터들을 활용하면서 최적화를 수행할 수 없음.

### Bayesian 최적화가 필요한 순간
- 가능한 최소의 시도로 최적의 답을 찾아야 할 경우
- 개별 시도가 너무 많은 시간/자원이 필요할 때

### 베이지안 최적화(Bayesian Optimization) 개요
- 베이지안 최적화는 미지의 함수가 반환하는 값의 최소 또는 최대값을 만드는 최적해를 짧은 반복을 통해 찾아내는 최적화 방식.
- 베이지안 최적화는 새로운 데이터를 입력 받았을 때 최적 함수를 예측하는 사후 모델을 개선해 나가면서 최적 함수를 도출
- 대체 모델(Surrogate Model)과 획득 함수로 구성되며, 대체 모델은 획득 함수로 부터 최적 입력 값을 추천 받은 뒤 이를 기반으로 최적 함수 모델을 개선
- 획득 함수는 개선된 대체 모델을 기반으로 다시 최적 입력 값을 계산

### 베이지안 최적화 수행 단계
1. 최초에는 랜덤하게 하이퍼 파라미터들을 샘플링하여 성능 결과를 관측
2. 관측된 값을 기반으로 대체 모델은 최적 함수를 예측 추정
3. 획득 함수에서 다음으로 관측할 하이퍼 파라미터 추출
4. 해당 하이퍼 파라미터로 관측된 값을 기반으로 대체 모델은 다시 최적 함수 예측 추정

### 베이지안 최적화 구현 요소
1. 입력 값 범위(Search_space = {"x" : (-10,10), "Y" : (-15,15)})
2. 함수
3. 함수 반환 최대/최소값 유추

### 베이지안 최적화를 구현한 주요 패키지
- HyperOpt
- Baesian optimization
- Optuna

##### 실습

In [1]:
import hyperopt

print(hyperopt.__version__)

0.2.7


In [2]:
from hyperopt import hp

search_space = {"x" : hp.quniform("x", -10, 10, 1), "y" : hp.quniform("y", -15, 15, 1)}

In [11]:
from hyperopt import STATUS_OK

# 목적 함수를 생성. 입력 변수값과 입력 변수 겁색 범위를 가지는 딕셔너리를 인자로 받고, 특정 값을 반환
def objective_func(search_space):
    x = search_space["x"]
    y = search_space["y"]
    retval  = x**2 - 20*y
    
    return retval

In [12]:
from hyperopt import fmin, tpe, Trials
import numpy as np

# 입력 결괏값을 저장한 Trials 객체값 생성.
trial_val = Trials()

# 목적 함수의 최솟값을 반환하는 최적 입력 변숫값을 5번의 입력값 시도(max_eval = 5)로 찾아냄
best_01 = fmin(fn = objective_func, space = search_space, algo = tpe.suggest, max_evals = 5,
              trials = trial_val, rstate = np.random.default_rng(seed = 0))
print("best : " ,best_01)

100%|█████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 313.27trial/s, best loss: -224.0]
best :  {'x': -4.0, 'y': 12.0}


In [13]:
trial_val = Trials()

# max_evals를 20회로 늘려서 재테스트
best_02 = fmin(fn=objective_func, space=search_space, algo=tpe.suggest, max_evals=20
               , trials=trial_val, rstate=np.random.default_rng(seed=0))
print('best:', best_02)

100%|███████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 392.40trial/s, best loss: -296.0]
best: {'x': 2.0, 'y': 15.0}


In [14]:
trial_val

- HyperOpt 수행 시 적용된 입력 값들과 목적 함수 반환값 보기

In [ ]:
### 이부분 설명 추가 

In [ ]:
### 이부분 설명 추가 

In [18]:
import pandas as pd

# results에서 loss 키 값에 해당하는 밸류들을 추출하여 list로 생성
losses = [loss_dict["loss"] for loss_dict in trial_val.results]

# DataFrame으로 생성.
result_df = pd.DataFrame({"x" : trial_val.vals["x"],
                         "y" : trial_val.vals["y"],
                         "losses" : losses})

result_df

,x,y,losses
0,-6.0,5.0,-64.0
1,-4.0,10.0,-184.0
2,4.0,-2.0,56.0
3,-4.0,12.0,-224.0
4,9.0,1.0,61.0
5,2.0,15.0,-296.0
6,10.0,7.0,-40.0
7,-9.0,-10.0,281.0
8,-8.0,0.0,64.0
9,-0.0,-5.0,100.0
